### 0.0 Download Required Libraries

In [ ]:
# %pip install pandas
# %pip install xlwt openpyxl xlsxwriter xlrd
# %pip install glob

### 1.0 Acquire User's Query Insights Data Set through API Request & Process Data
#### 1.1 Configure Epoch Time for Specific Data Range

In [1]:
import datetime

# configure start date and re: epoch time using datetime library
start_date_time = datetime.datetime(2024, 2, 22, 13, 48, 41)
start_epoch_time = int(round(start_date_time.timestamp()))

# configure end date and re: epoch time using datetime library
end_date_time = datetime.datetime(2024, 2, 28, 13, 48, 40)
end_epoch_time = int(round(end_date_time.timestamp()))

# output start and end date for troubleshooting
print(start_date_time)
print(end_date_time)

2024-02-22 13:48:41
2024-02-28 13:48:40


#### 1.2 Configure API GET Req for Campaign

In [ ]:
# import libraries
import requests
import json
import pandas as pd

# url for user dialogue data set
dial_url = 'https://h10.company.com/api/v1/agent/2000/query_insights.json'

# credentials for accessing content
headers = {
    "Content-Type": "application/json",
    "access-token": '12345678905bfc72b1234000jht4451234567890'
}

# submit GET request
dialogue_r = requests.get(dial_url, headers=headers, 
                          params={'start_time': start_epoch_time, 'end_time': end_epoch_time, 'offset': -18000, "page": 1, "per_page": 100})


# Raise exception for different HTTP response conditions
dialogue_r.raise_for_status()

# output status code for GET request
print(dialogue_r.status_code)

# store convert response to json format 
dial_response = dialogue_r.json()

# output response in more legible json format
print(json.dumps(dial_response, indent=4))

#### 1.3 Extract Key Insights from Key-Value Pairs

In [ ]:
# extract entries key-value pairs
queries = dial_response['queries']
qi_users = []
qi_custom_prop = []
qi_delivery_status = []

# extract user parent data
for index in queries:
    qi_users.append(index['user'])

# pull user data into query insights DataFrame
qi_users_df = pd.DataFrame(qi_users)

# loop through user key-pair to extract custom_property elements
for index in qi_users:
    qi_custom_prop.append(index['custom_properties'])

# extract custom properties from users to acquire specific elements
qi_custom_prop_df = pd.DataFrame(qi_custom_prop)

# expand view to contain all columns in DataFrame
pd.set_option('display.max_columns', None)

# output sample size of custom properties DataFrame
qi_custom_prop_df.head()

### 2.0 Acquire User's Outreach Insights Data Set through API Request & Process Data
#### 2.1 Configure API GET Req for Campaign

In [ ]:
import requests

# url for user outreach data set
base_url = "https://h10.company.com/api/v2/campaigns/insights"

# manual endpoint creation
campaign_id = 200
# campaign_query_url = base_url + '?campaign_id=' + f"{campaign_id}"

execution_query = '?execution_id='
execution_id = 2000
# campaign_stats_url = campaign_query_url + execution_query + f'{execution_id}'

# validation of user credentials
headers = {
    "Content-Type": "application/json",
    "access-token": '12345678905bfc72b1afe1234567890'
}

# # get request w/ automated endpoint creation & pagination
r = requests.get(base_url, headers=headers, params = { 'campaign_id' : campaign_id, "execution_id": execution_id,
                                                       "page": 1, "per_page": 100})

print(r.status_code)

#### 2.2 Verify & Evaluate Data Pull 

In [ ]:
import pandas as pd

# convert response to json format in new var 
info = r.json()
print(len(info))

# output data in legible json format
print(json.dumps(info, indent=4))

# create metrics to be extracted 
entries = info['entries']
users = []
campaign_name = []
delivery_status = []
campaign_filters = []

# extract values for all user entries
for i in entries:
    users.append(i['user'])

# extract campaign name for extract
campaign_name = entries[0]['filters'][4]['name']

# extract delivery_status for all entries
for i in entries:
    delivery_status.append(i['delivery_status'])

# extract campaign details from entries list
for index in entries:
    campaign_filters.append(i['filters'])


# create dataframes with required elements
users_metrics =         pd.DataFrame(users)
delivery_metrics_df =   pd.DataFrame(delivery_status)
campaign_filters_df =   pd.DataFrame(campaign_filters)

# extract patients part of outreach campaign 
outreach_metrics_col = []

# pull insights from specific col in DataFrame into new list
for index in campaign_filters_df[4]:
    outreach_metrics_col.append({'outreach_metrics': index['match']})

# convert list into new DataFrames
practice = pd.DataFrame(outreach_metrics_col)
users_metrics = pd.concat([users_metrics, practice], axis=1)

# output users_metrics into DataFrame
users_metrics